<a href="https://colab.research.google.com/github/Wozny614/mispy/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sformułowanie problemu


Zadanie polega na stworzeniu modelu neurowego, który na podstawie dostępnych danych potrafi określić która, osoba byłaby w stanie przetrwać katastrofę Titanic'a. Dostępne dane to:

|Variable|Definition|Key
|-|-|-
|Survival|Surivial|0=No, 1=Yes
|pclass|Ticket class|1=1st, 2=2nd,3=3rd
|sex|Sex|-
|Age|Age in years|-
|sibsip|# of siblings/spouses abroad the Titanic|-
|parch|# of parents/children abroad the Titanic|-
|ticket|Ticket number|-
|fare|Passenger fare|-
|cabin|Cabin number|-
|embarked|Port of Embarkation|C=Cherbourg, Q=Queenstown, S=Southampton

Plik *train.csv* zawiera:


In [88]:
pd.read_csv('/content/train.csv').head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Natomiast plik *test.csv* zawiera:

In [89]:
pd.read_csv('/content/test.csv').head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Jak widać nie posiada on informacji o tym czy dany pasażer przeżył czy nie.
Określenie tego jest właśnie zadaniem sieci.

# Rozwiązanie


In [90]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [91]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
full = train.append(test, ignore_index=True)


In [92]:
full.describe()     # the information of only numbers
full.info()         # the information of all variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


Niektóre komórki mogą być puste, dlatego przypisujemy im wartość średnią z całego zbioru.

In [93]:
full['Age'] = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
full['Embarked'] = full['Embarked'].fillna('S')
full['Cabin'] = full['Cabin'].fillna('U')       # "Unknown"

Przepisanie "male"/"female" na wartości liczbowe 1,0

In [94]:
sex_mapDict = {'male': 1, 'female': 0}
full['Sex'] = full['Sex'].map(sex_mapDict)

In [95]:
embarkedDF = pd.get_dummies(full['Embarked'], prefix='Embarked')
embarkedDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, embarkedDF, on='PassengerId')

In [96]:


PclassDF = pd.get_dummies(full['Pclass'], prefix='Pclass')
PclassDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, PclassDF, on='PassengerId')



Przydzielenie odpowiednio tytułu do "klasy społecznej"


In [97]:
def getTitle(name):
    str1 = name.split(',')[1]
    str2 = str1.split('.')[0]
    str3 = str2.strip()
    return str3

titleDF = pd.DataFrame()
titleDF['Title'] = full['Name'].map(getTitle)
title_mapDict = {"Capt":         "Officer",
                 "Col":          "Officer",
                 "Major":        "Officer",
                 "Jonkheer":     "Royalty",
                 "Don":          "Royalty",
                 "Sir":          "Royalty",
                 "Dr":           "Officer",
                 "Rev":          "Officer",
                 "the Countess": "Royalty",
                 "Dona":         "Royalty",
                 "Mme":          "Mrs",
                 "Mlle":         "Miss",
                 "Ms":           "Mrs",
                 "Mr":           "Mr",
                 "Mrs":          "Mrs",
                 "Miss":         "Miss",
                 "Master":       "Master",
                 "Lady":         "Royalty"}
titleDF['Title'] = titleDF['Title'].map(title_mapDict)
titleDF = pd.get_dummies(titleDF['Title'])
titleDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, titleDF, on='PassengerId')

In [98]:
cabinDF = pd.DataFrame()
cabinDF['Cabin'] = full['Cabin'].map(lambda c: c[0])
cabinDF = pd.get_dummies(cabinDF['Cabin'], prefix='Cabin')
cabinDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, cabinDF, on='PassengerId')

In [99]:
familyDF = pd.DataFrame()
familyDF['FamilySize'] = full['Parch'] + full['SibSp'] + 1
familyDF['Family_Single'] = familyDF['FamilySize'].map(lambda s: 1 if (s == 1) else 0)
familyDF['Family_Small'] = familyDF['FamilySize'].map(lambda s: 1 if (2 <= s <= 4) else 0)
familyDF['Family_Large'] = familyDF['FamilySize'].map(lambda s: 1 if (s >= 5) else 0)
familyDF['PassengerId'] = full['PassengerId']
full = pd.merge(full, familyDF, on='PassengerId')

In [100]:
corrDF = full.corr()
print(corrDF['Survived'].sort_values(ascending=False))
full_X = full[['Mrs', 'Miss', 'Pclass_1', 'Family_Small', 'Fare', 'Family_Single', 'Pclass_3', 'Sex', 'Mr']]

Survived         1.000000
Mrs              0.344935
Miss             0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Master           0.085221
Parch            0.081629
Cabin_F          0.057935
Royalty          0.033391
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Officer         -0.031316
SibSp           -0.035322
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Pclass          -0.338481
Sex             -0.543351
Mr              -0.549199
Name: Survived, dtype: float64


In [101]:
sourceRow = full['Survived'].count()
source_X = full_X.loc[0:sourceRow-1, :]
source_y = full.loc[0:sourceRow-1, 'Survived']
predict_X = full_X.loc[sourceRow:, :]

In [102]:
train_X, test_X, train_y, test_y = train_test_split(source_X, source_y, train_size=.8)
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(train_X, train_y)
print(model.score(test_X, test_y))

0.7877094972067039


In [103]:
predict_y = model.predict(predict_X).astype(int)
passenger_Id = full.loc[sourceRow:, 'PassengerId']
predictDF = pd.DataFrame({'PassengerId': passenger_Id,
                          'Survived': predict_y})
predictDF.to_csv('prediction1.csv', index=False)

In [104]:
pd.read_csv('/content/prediction1.csv')

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
